In [1]:
import os
import cv2
import sys
import random
import math
import re
import time
import numpy as np
import tensorflow as tf
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import skimage
import glob
import datetime
# Import Mask RCNN
working_dir = os.getcwd()
print("Working directory is:",working_dir)
sys.path.append(working_dir)  # To find local version of the library
from mrcnn import utils
from mrcnn import visualize
from mrcnn.visualize import display_images
import mrcnn.model as modellib
from mrcnn.model import log

/home/user/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Working directory is: /home/user/Source/Evrim/Turkishplate


Using TensorFlow backend.


In [2]:


%matplotlib inline 

custom_WEIGHTS_PATH = "mask_rcnn_plate_0010.h5"  # TODO: update this path

# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

#GPU MACHINE
ROOT_DIR= "/home/user/Source/Evrim/Turkishplate"
#LOCAL
#ROOT_DIR= "/Users/burke.atilla"

sys.path.append(ROOT_DIR + "/Python")
import custom 

# Root directory of the project
ROOT_DATA_DIR = "/home/user/Source/data/dataEvrim/Original_15K"
# Directory to save logs and trained model
MODEL_DIR= '/home/user/Source/Evrim/Turkishplate/Model'
print("ROOT_DATA_DIR set to:",ROOT_DATA_DIR)

config = custom.CustomConfig()
custom_DIR = os.path.join(ROOT_DATA_DIR, "originals")
print("custom_DIR set to:",custom_DIR)

print("MODEL_DIR set to:",MODEL_DIR)




ROOT_DATA_DIR set to: /home/user/Source/data/dataEvrim/Original_15K
custom_DIR set to: /home/user/Source/data/dataEvrim/Original_15K/originals
MODEL_DIR set to: /home/user/Source/Evrim/Turkishplate/Model


In [3]:
# Override the training configurations with a few
# changes for inferencing.
class InferenceConfig(config.__class__):
    # Run detection on one image at a time
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

config = InferenceConfig()
config.display()


Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.9
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                14
IMAGE_MIN_DIM                  800
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE                     [28, 28]
MAX_GT_INSTA

In [4]:
# Device to load the neural network on.
# Useful if you're training a model on the same 
# machine, in which case use CPU and leave the
# GPU for training.
DEVICE = "/cpu:0"  # /cpu:0 or /gpu:0
#DEVICE = "/cpu:0"  # /cpu:0 or /gpu:0

# Inspect the model in training or inference modes
# values: 'inference' or 'training'
# TODO: code for 'training' test mode not ready yet
TEST_MODE = "inference"

In [5]:
def get_ax(rows=1, cols=1, size=16):
    """Return a Matplotlib Axes array to be used in
    all visualizations in the notebook. Provide a
    central point to control graph sizes.
    
    Adjust the size attribute to control how big to render images
    """
    _, ax = plt.subplots(rows, cols, figsize=(size*cols, size*rows))
    return ax

In [ ]:
# Create model in inference mode
with tf.device(DEVICE):
    localize_model = modellib.MaskRCNN(mode="inference", model_dir=MODEL_DIR,config=config)
    
# load the last model you trained
# weights_path = model.find_last()[1]
start_time = datetime.datetime.now()

# Load weights
print("Loading weights ", MODEL_DIR)
localize_model.load_weights(MODEL_DIR + "/mask_rcnn_plate_0010.h5", by_name=True)

end_time = datetime.datetime.now()
print (start_time - start_time)
        
start_time = datetime.datetime.now()
from scipy.ndimage import rotate



Loading weights  /home/user/Source/Evrim/Turkishplate/Model
0:00:00


In [ ]:
from skimage import io
import shutil

inp_img_path = '/home/user/Source/data/dataEvrim/Original_15K/'
out_img_path = '/home/user/Source/data/dataEvrim/Original_15Kcrop/'
moved_img_path = '/home/user/Source/data/dataEvrim/Original_15Kmoved/'
cnt=0

for file in sorted(os.listdir(inp_img_path)):    
    image = skimage.io.imread(inp_img_path + file)    
    results = localize_model.detect([image], verbose=1)
    # Display results
    #ax = get_ax(1)
    r = results[0]
    try:
        x1 = r['rois'][0][0]
        y1 = r['rois'][0][1]
        x2 = r['rois'][0][2]
        y2 = r['rois'][0][3]  
    except:
        print("error infilename:", file)
        
    io.imsave(out_img_path + file , image[x1:x2,y1:y2]  )
    shutil.move(inp_img_path +  file , moved_img_path )
    cnt +=1
    
print("toplam işlenen dosya1 :" , cnt)



Processing 1 images
image                    shape: (720, 1280, 3)        min:    0.00000  max:  255.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  151.10000  float64
image_metas              shape: (1, 14)               min:    0.00000  max: 1280.00000  float64
anchors                  shape: (1, 261888, 4)        min:   -0.35390  max:    1.29134  float32
Processing 1 images
image                    shape: (720, 1280, 3)        min:    0.00000  max:  255.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  150.10000  float64
image_metas              shape: (1, 14)               min:    0.00000  max: 1280.00000  float64
anchors                  shape: (1, 261888, 4)        min:   -0.35390  max:    1.29134  float32
Processing 1 images
image                    shape: (720, 1280, 3)        min:    0.00000  max:  255.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  151.1

Processing 1 images
image                    shape: (720, 1280, 3)        min:    0.00000  max:  255.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  151.10000  float64
image_metas              shape: (1, 14)               min:    0.00000  max: 1280.00000  float64
anchors                  shape: (1, 261888, 4)        min:   -0.35390  max:    1.29134  float32
Processing 1 images
image                    shape: (720, 1280, 3)        min:    0.00000  max:  255.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  151.10000  float64
image_metas              shape: (1, 14)               min:    0.00000  max: 1280.00000  float64
anchors                  shape: (1, 261888, 4)        min:   -0.35390  max:    1.29134  float32
Processing 1 images
image                    shape: (720, 1280, 3)        min:    0.00000  max:  255.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  151.1

Processing 1 images
image                    shape: (720, 1280, 3)        min:    0.00000  max:  255.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  151.10000  float64
image_metas              shape: (1, 14)               min:    0.00000  max: 1280.00000  float64
anchors                  shape: (1, 261888, 4)        min:   -0.35390  max:    1.29134  float32
Processing 1 images
image                    shape: (720, 1280, 3)        min:    0.00000  max:  255.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  151.10000  float64
image_metas              shape: (1, 14)               min:    0.00000  max: 1280.00000  float64
anchors                  shape: (1, 261888, 4)        min:   -0.35390  max:    1.29134  float32
Processing 1 images
image                    shape: (720, 1280, 3)        min:    0.00000  max:  255.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  150.1

Processing 1 images
image                    shape: (720, 1280, 3)        min:    0.00000  max:  255.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  151.10000  float64
image_metas              shape: (1, 14)               min:    0.00000  max: 1280.00000  float64
anchors                  shape: (1, 261888, 4)        min:   -0.35390  max:    1.29134  float32
Processing 1 images
image                    shape: (720, 1280, 3)        min:    0.00000  max:  255.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  151.10000  float64
image_metas              shape: (1, 14)               min:    0.00000  max: 1280.00000  float64
anchors                  shape: (1, 261888, 4)        min:   -0.35390  max:    1.29134  float32
Processing 1 images
image                    shape: (720, 1280, 3)        min:    0.00000  max:  255.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  151.1

In [ ]:
from skimage import io
import shutil

inp_img_path = '/home/user/Source/data/dataEvrim/Original_15Kb/'
out_img_path = '/home/user/Source/data/dataEvrim/Original_15Kcrop/'
moved_img_path = '/home/user/Source/data/dataEvrim/Original_15Kmoved/'
cnt=0

for file in sorted(os.listdir(inp_img_path)):    
    image = skimage.io.imread(inp_img_path + file)    
    results = localize_model.detect([image], verbose=1)
    # Display results
    #ax = get_ax(1)
    r = results[0]
    try:
        x1 = r['rois'][0][0]
        y1 = r['rois'][0][1]
        x2 = r['rois'][0][2]
        y2 = r['rois'][0][3]  
    except:
        print("error infilename:", file)
        
    io.imsave(out_img_path + file , image[x1:x2,y1:y2]  )
    shutil.move(inp_img_path +  file , moved_img_path )
    cnt +=1
    
print("toplam işlenen dosya1 :" , cnt)

In [ ]:
#PLATE DETECTION

In [ ]:
"""

import keras
import tensorflow as tf
import sys
import os
from keras import backend as K
import itertools

sess = tf.Session()
K.set_session(sess)

#sys.path.append(os.path.dirname(os.path.realpath('Textgen')) + "/python")
import Textgen as t

#load model
loaded_model=t.load_model()
"""

In [ ]:
"""

letters_train = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'R', 'S', 'T', 'U', 'V', 'Y', 'Z']
letters = sorted(list(letters_train))

def decode_batch(out):
    ret = []
    for j in range(out.shape[0]):
        out_best = list(np.argmax(out[j, 2:], 1))
        out_best = [k for k, g in itertools.groupby(out_best)]
        outstr = ''
        for c in out_best:
            if c < len(letters):
                outstr += letters[c]
        ret.append(outstr)
    return ret

def predict_single_plate(sess,loaded_model,img):
    img_w,img_h = 128, 64
   
    #img = cv2.imread(img_filepath)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img = cv2.resize(img, (img_w, img_h))
    img = img.astype(np.float32)
    img /= 255


    net_inp = loaded_model.get_layer(name='the_input').input
    net_out = loaded_model.get_layer(name='softmax').output

    if K.image_data_format() == 'channels_first':
        X_data = np.ones([1, 1, img_w, img_h])
    else:
        X_data = np.ones([1, img_w, img_h, 1])
    
    img = img.T
    if K.image_data_format() == 'channels_first':
        img = np.expand_dims(img, 0)
    else:
        img = np.expand_dims(img, -1)
    X_data[0] = img

    net_out_value = sess.run(net_out, feed_dict={net_inp:X_data})
    pred_texts = decode_batch(net_out_value)
    print(pred_texts)
"""

In [ ]:
"""

#predict plate
predicted_text = predict_single_plate(sess,loaded_model,image[x1:x2,y1:y2])#input directory of pic
print(predicted_text)
"""